In [15]:
import os
import pickle
from utils.b2 import B2
from utils.modeling import *
import streamlit as st
from dotenv import load_dotenv
import numpy as np
# from implicit.als import AlternatingLeastSquares
# from sklearn.metrics import mean_squared_error
# # from scipy.sparse import coo_matrix
# # import category_encoders as ce
# import pandas as pd
#import pymc as pm
# from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, balanced_accuracy_score
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

ImportError: cannot import name 'SettingWithCopyWarning' from 'pandas.core.common' (c:\Users\Minh\miniconda3\envs\i501-project\lib\site-packages\pandas\core\common.py)

In [ ]:
# ------------------------------------------------------
#                      APP CONSTANTS
# ------------------------------------------------------
REMOTE_DATA = 'pbp.csv'

In [ ]:
# ------------------------------------------------------
#                        CONFIG
# ------------------------------------------------------
load_dotenv()

True

In [ ]:
# load Backblaze connection
b2 = B2(endpoint=os.environ['B2_ENDPOINT'],
        key_id=os.environ['B2_KEYID'],
        secret_key=os.environ['B2_APPKEY'])

In [ ]:
# ------------------------------------------------------
#                        CACHING
# ------------------------------------------------------
@st.cache_data
def get_data():
    # collect data frame of reviews and their sentiment
    b2.set_bucket(os.environ['B2_BUCKETNAME'])
    df_pbp = b2.get_df(REMOTE_DATA)

    return df_pbp

2024-04-10 14:42:05.399 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [ ]:
data = get_data()

2024-04-10 14:42:05.777 
  command:

    streamlit run c:\Users\Minh\miniconda3\envs\i501-project\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-04-10 14:42:05.779 No runtime found, using MemoryCacheStorageManager


In [ ]:
@st.cache_data
def prep_data(data):
    rush_data = data[data['IsRush'] == 1]
    pass_data = data[data['IsPass'] == 1]
    # Train pass play model
    X_pass = pass_data[['SitID', 'PlayID']]
    y_pass = pass_data['Yards']
    # Train rush play model
    X_rush = rush_data[['SitID', 'PlayID']]
    y_rush = rush_data['Yards']
    return X_pass, y_pass, X_rush, y_rush, rush_data, pass_data

2024-04-10 14:42:08.564 No runtime found, using MemoryCacheStorageManager


In [ ]:
X_pass, y_pass, X_rush, y_rush, rush_data, pass_data = prep_data(data)

2024-04-10 14:42:08.581 No runtime found, using MemoryCacheStorageManager


In [ ]:
@st.cache_data
def get_model(X_pass, y_pass, X_rush, y_rush, rush_data, pass_data):
    pass_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
    pass_regressor.fit(X_pass, y_pass)
    rush_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
    rush_regressor.fit(X_rush, y_rush)
    rush_data.loc[:, 'Predicted Yards'] = rush_regressor.predict(X_rush)
    best_rush_plays = rush_data.sort_values(by='Predicted Yards', ascending=False)
    pass_data.loc[:, 'Predicted Yards'] = pass_regressor.predict(X_pass)
    best_pass_plays = pass_data.sort_values(by='Predicted Yards', ascending=False)
    return best_rush_plays, best_pass_plays

2024-04-10 14:42:08.688 No runtime found, using MemoryCacheStorageManager


In [ ]:
best_rush_plays, best_pass_plays = get_model(X_pass, y_pass, X_rush, y_rush, rush_data, pass_data)

2024-04-10 14:42:08.704 No runtime found, using MemoryCacheStorageManager
C:\Users\Minh\AppData\Local\Temp\ipykernel_18364\2060931429.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rush_data.loc[:, 'Predicted Yards'] = rush_regressor.predict(X_rush)
C:\Users\Minh\AppData\Local\Temp\ipykernel_18364\2060931429.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pass_data.loc[:, 'Predicted Yards'] = pass_regressor.predict(X_pass)


In [ ]:
@st.cache_data
def getPlay(team, down, distance, yardline):
    teamId = data.loc[data['OffenseTeam'] == team, 'TeamID'].values[0]
    SitId = yardline + 100 * distance + 10000 * down + 100000 * teamId
    SitId = 1711083
    if SitId in best_pass_plays['SitID'].values:
        best_passes = best_pass_plays[best_pass_plays['SitID'] == SitId]
        best_passes = best_passes.nlargest(2, 'Predicted Yards')
        print('First Pass Choice: ', best_passes.iloc[0]['PassType'])
        print('Predicted Gain: ', round(float(best_passes.iloc[0]['Predicted Yards'])))
        print('Second Pass Choice: ', best_passes.iloc[1]['PassType'])
        print('Predicted Gain: ', round(float(best_passes.iloc[1]['Predicted Yards'])))

    if SitId in best_rush_plays['SitID'].values:
        best_rushes = best_rush_plays[best_rush_plays['SitID'] == SitId]
        best_rushes = best_rushes.nlargest(2, 'Predicted Yards')
        print('First Rush Choice: ', best_rushes.iloc[0]['RushDirection'])
        print('Predicted Gain: ', round(float(best_rushes.iloc[0]['Predicted Yards'])))
        print('Second Rush Choice: ', best_rushes.iloc[1]['RushDirection'])
        print('Predicted Gain: ', round(float(best_rushes.iloc[1]['Predicted Yards'])))

2024-04-10 14:42:20.781 No runtime found, using MemoryCacheStorageManager


In [ ]:
getPlay('NE', 1, 10, 20)

2024-04-10 14:42:20.798 No runtime found, using MemoryCacheStorageManager


First Pass Choice:  DEEP LEFT
Predicted Gain:  76
Second Pass Choice:  SHORT MIDDLE
Predicted Gain:  4
First Rush Choice:  LEFT END
Predicted Gain:  8
Second Rush Choice:  LEFT TACKLE
Predicted Gain:  5
